In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, ForeignKey, Date
from sqlalchemy.orm import relationship
import pandas as pd

In [2]:
def load_lifts_csv(csv_file: str) -> pd.DataFrame:
    """
    Load lifts .csv file to a DataFrame
    """
    # fields: date, exercise, category, weight, reps
    df = pd.read_csv(csv_file)

    # drop useless columns, change column types, etc.
    df = df.drop("Distance", 1)
    df = df.drop("Distance Unit", 1)
    df = df.drop("Time", 1)
    df["Date"] = pd.to_datetime(df["Date"])
    # df["Comment"] = df["Comment"].astype("object")
    df = df.drop("Comment", 1)

    # create 1RM column
    df["1RM"] = df["Weight (lbs)"] * 1.03 ** (df["Reps"] - 1)
    
    df = df.rename(columns = {"Weight (lbs)": "weight", "Date": "date", "Exercise": "exercise", "Category": "category", "Weight": "weight", "Reps": "reps", "1RM": "orm"})

    return df

df = load_lifts_csv(
        r"C:\Users\andre\Downloads\FitNotes_Export_2021_01_03_16_31_46.csv"
    )

df.head(10)

,date,exercise,category,weight,reps,orm
0,2015-12-26,Seated Barbell Press,Shoulders,44.09,5.0,49.623683
1,2015-12-26,Seated Barbell Press,Shoulders,44.09,5.0,49.623683
2,2015-12-26,Seated Barbell Press,Shoulders,88.18,5.0,99.247367
3,2015-12-26,Seated Barbell Press,Shoulders,88.18,5.0,99.247367
4,2015-12-26,Seated Barbell Press,Shoulders,88.18,5.0,99.247367
5,2015-12-26,Deadlift,Back,99.21,5.0,111.661729
6,2015-12-26,Deadlift,Back,110.23,5.0,124.064836
7,2015-12-26,Pull Up,Back,0.00,3.0,0.000000
8,2015-12-27,Flat Barbell Bench Press,Chest,44.09,5.0,49.623683
9,2015-12-27,Flat Barbell Bench Press,Chest,44.09,5.0,49.623683


In [3]:

engine = create_engine("sqlite:///C:\\Development\\lifting-tracker\\lift_tracker.db", echo = True)

Base = declarative_base()

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True)
    username = Column("username", String(32))
    lifts = relationship("Lift")
    bodies = relationship("Body")


class Body(Base):
    __tablename__ = "bodies"
    id = Column(Integer, primary_key=True)
    date = Column("date", Date)
    measurement = Column("measurement", String(64))
    value = Column("value", Numeric)
    unit = Column("unit", String(32))
    user_id = Column(Integer, ForeignKey("users.id"))


class Lift(Base):
    __tablename__ = "lifts"
    id = Column(Integer, primary_key=True)
    exercise = Column("exercise", String(64))
    category = Column("category", String(32))
    weight = Column("weight", Numeric)
    reps = Column("reps", Integer)
    orm = Column("orm", Numeric)
    date = Column("date", Date)
    user_id = Column(Integer, ForeignKey("users.id"))
    
Base.metadata.create_all(engine)

from sqlalchemy.orm import sessionmaker
import datetime
Session = sessionmaker(bind = engine)
session = Session()

2021-01-03 21:54:05,107 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-01-03 21:54:05,111 INFO sqlalchemy.engine.base.Engine ()
2021-01-03 21:54:05,113 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-01-03 21:54:05,114 INFO sqlalchemy.engine.base.Engine ()
2021-01-03 21:54:05,115 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2021-01-03 21:54:05,116 INFO sqlalchemy.engine.base.Engine ()
2021-01-03 21:54:05,119 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2021-01-03 21:54:05,120 INFO sqlalchemy.engine.base.Engine ()
2021-01-03 21:54:05,121 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("bodies")
2021-01-03 21:54:05,123 INFO sqlalchemy.engine.base.Engine ()
2021-01-03 21:54:05,125 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("bodies")
2021-01-03 21:54:05,127 INFO sqlalchemy.engine.base.Engine ()
2021-01-03 21:54:05,129 

In [4]:
df.to_sql("lifts", engine, if_exists = "append", index = False)

2021-01-03 21:54:15,349 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("lifts")
2021-01-03 21:54:15,349 INFO sqlalchemy.engine.base.Engine ()
2021-01-03 21:54:15,375 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-01-03 21:54:15,725 INFO sqlalchemy.engine.base.Engine INSERT INTO lifts (date, exercise, category, weight, reps, orm) VALUES (?, ?, ?, ?, ?, ?)
2021-01-03 21:54:15,733 INFO sqlalchemy.engine.base.Engine (('2015-12-26 00:00:00.000000', 'Seated Barbell Press', 'Shoulders', 44.09, 5.0, 49.62368343290001), ('2015-12-26 00:00:00.000000', 'Seated Barbell Press', 'Shoulders', 44.09, 5.0, 49.62368343290001), ('2015-12-26 00:00:00.000000', 'Seated Barbell Press', 'Shoulders', 88.18, 5.0, 99.24736686580002), ('2015-12-26 00:00:00.000000', 'Seated Barbell Press', 'Shoulders', 88.18, 5.0, 99.24736686580002), ('2015-12-26 00:00:00.000000', 'Seated Barbell Press', 'Shoulders', 88.18, 5.0, 99.24736686580002), ('2015-12-26 00:00:00.000000', 'Deadlift', 'Back', 99.21, 5.0,

In [ ]:
c1 = Lift(exercise = "Flat Barbell Bench Press", category = "Chest", weight = 290, reps = 1, orm = "290", date = datetime.date(2020, 12, 29), user_id = 1)

session.add(c1)
session.commit()

In [ ]:
result = session.query(Lift).first()

for row in result:
    print("Name: ",row.exercise, "Weight:",row.weight, "Reps:",row.reps)

In [5]:
session.close()

In [4]:
session.query(Lift).delete()

2021-01-03 21:49:17,948 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-01-03 21:49:17,951 INFO sqlalchemy.engine.base.Engine DELETE FROM lifts
2021-01-03 21:49:17,953 INFO sqlalchemy.engine.base.Engine ()


17275

In [ ]:
db_cols = Lift.__table__.columns.keys()
df_cols = df.columns.tolist()

set(db_cols) - set(df_cols)